# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 3.0.5 which is incompatible.


## Set up initial environment
The cell below instantiates the environment and sets some initial variables:

- brain_name
- action_size: the number of actions that can be performed in the environment
- state_size: the number of values retured from the envionment to represent the current state

In [2]:
from unityagents import UnityEnvironment
import numpy as np

# select this option to load version 1 (with a single agent) of the environment
env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64')

# select this option to load version 2 (with 20 agents) of the environment
# env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000e+00
   1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   5.75471878e+00  -1.00000000e+00
   5.55726671e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
  -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [3]:
env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
counter = 0
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    counter += 1
    #print('dones: {}'.format(dones))
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))
print('max episode length: {}'.format(counter))


Total score (averaged over agents) this episode: 0.0
max episode length: 1001


### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

In [4]:
# define netowrks
import torch
import torch.nn.functional as F
import torch.optim as optim
import copy
import random

def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)

# actor - take in a state and output a distribution of actions
class ActorModel(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(ActorModel, self).__init__()
        self.state_size   = state_size
        self.action_size = action_size

        self.fc1 = torch.nn.Linear(state_size, 256)
        self.fc2 = torch.nn.Linear(256, 256)
        self.out = torch.nn.Linear(256, action_size)
        # self.reset_parameters()

    def reset_parameters(self):
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.out.weight.data.uniform_(-3e-3, 3e-3)
        
    def forward(self, states):
        batch_size = states.size(0)
        x = F.relu(self.fc1(states))
        x = F.relu(self.fc2(x))
        x = F.tanh(self.out(x))
        return x

# critic - take in a state AND actions - outputs a state value function - V
class CriticModel(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(CriticModel, self).__init__()
        self.state_size   = state_size
        self.action_size = action_size

        self.fc1 = torch.nn.Linear(state_size, 256)
        self.fc2 = torch.nn.Linear(256+action_size, 256)
        self.fc3 = torch.nn.Linear(256, 128)
        self.out = torch.nn.Linear(128, 1)
        # self.reset_parameters()

    def reset_parameters(self):
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(*hidden_init(self.fc3))
        self.out.weight.data.uniform_(-3e-3, 3e-3)
        
    def forward(self, states, actions):
        batch_size = states.size(0)
        xs = F.leaky_relu(self.fc1(states))
        x = torch.cat((xs, actions), dim=1) #add in actions to the network
        x = F.leaky_relu(self.fc2(x))
        x = F.leaky_relu(self.fc3(x))
        x = self.out(x)
        return x
    
class StepInfo:
    def __init__(self, step_number, states, actions, rewards, next_states, dones):
        self.step_number = step_number
        self.states = states
        self.actions = actions
        self.rewards = rewards
        self.next_states = next_states
        self.dones = dones
        
    def __str__(self):
        return "step_number: {},  states: {},  actions: {},  rewards: {},  next_states: {}".format(self.step_number, self.states, self.actions, self.rewards, self.next_states)

class OUNoise:
    """Ornstein-Uhlenbeck process."""

    def __init__(self, size, seed, mu=0., theta=0.15, sigma=0.2):
        """Initialize parameters and noise process."""
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.seed = random.seed(seed)
        self.reset()

    def reset(self):
        """Reset the internal state (= noise) to mean (mu)."""
        self.state = copy.copy(self.mu)

    def sample(self):
        """Update internal state and return it as a noise sample."""
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.array([random.random() for i in range(len(x))])
        self.state = x + dx
        return self.state

def reset_game(in_env, brain_name):
    env_info = in_env.reset(train_mode=True)[brain_name]      # reset the environment    
    states = env_info.vector_observations
    return states

def env_step(in_env, brain_name, states, replay_buffer, actor_model, epsilon, add_noise=True):
    #Play a game. Add to the replay_buffer
    if (len(replay_buffer) > 0):
        step_number = replay_buffer[-1].step_number + 1
    else:
        step_number = 0

    state_tensor = torch.from_numpy(states).float().cuda()

    rand_num = random.uniform(0, 1)
    if rand_num > epsilon:
        actor_model.eval()
        with torch.no_grad():
            actions_tensor = actor_model(state_tensor)
        actor_model.train()
        actions_np = actions_tensor.detach().cpu().numpy()
    else:
        actions_np = np.random.randn(num_agents, action_size)  # select an action (for each agent)

    env_info = in_env.step(actions_np)[brain_name]         # send all actions to tne environment
    next_states = env_info.vector_observations          # get next state (for each agent)
    rewards = env_info.rewards                          # get reward (for each agent)
    #print("rewards: {}".format(rewards))
    dones = env_info.local_done                         # see if episode finished

    this_step_info = StepInfo(step_number, states, actions_np, rewards, next_states, dones)
    replay_buffer.append(this_step_info)

    return next_states
            
def soft_update_target(local_model, target_model, tau):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
        target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

In [5]:
# training process (A3C): 
# Feed current_state into actor network to get the action to take in that state
#     Get next_state and rewards from the env

# Feed next_state (s-prime) into critic network to get critic_next_state_reward
#     Train the critic for current state using: current_reward + gamma * critic_next_state_reward

# Calculate Advantage (A) given current_state and action:
#     A(current_state, action) = reward + gamma(critic_next_state_reward) - critic_current_state_reward

# Train the Actor using the Calculated Advantage (A) as a baseline

# N-step bootstrapping???  Lambda (Generalized Advantage Estimate)???
# no replay buffer in AC3
# ************** not sure A3C will work with continuous action spaces

# training process (DDPG) - continuous action spaces
# Query the actor network to get believed best action (BBA)
# The critic evaluates the actor's BBA Given state AND actor's action values as input.
# DDPG uses a replay buffer
# Need a regular and target network for the actor and critic networks.
#  update targets using soft update strategy - e.g. slowly mix in updates.
#  target is used for predictions.


In [6]:
from collections import deque

# instantiate objects that will can be re-used
buffer_length = 100000
replay_buffer = deque(maxlen=buffer_length)

actor_model_local   = ActorModel(state_size, action_size).cuda()
actor_model_target  = ActorModel(state_size, action_size).cuda()
critic_model_local  = CriticModel(state_size, action_size).cuda()
critic_model_target = CriticModel(state_size, action_size).cuda()

if True:
    actor_model_local.load_state_dict(torch.load("actor_model_local.pt"))
    actor_model_target.load_state_dict(torch.load("actor_model_target.pt"))
    critic_model_local.load_state_dict(torch.load("critic_model_local.pt"))
    critic_model_target.load_state_dict(torch.load("critic_model_target.pt"))

lr_actor = .0002
lr_critic = .0002
weight_decay = 0.0
actor_optimizer = optim.Adam(actor_model_local.parameters(), lr=lr_actor)
critic_optimizer = optim.Adam(critic_model_local.parameters(), lr=lr_critic, weight_decay=weight_decay)
# random_seed = 1
# noise = OUNoise(actor_model_local.action_size, random_seed)

In [7]:
def getBatch(replay_buffer, batch_size):
    return_states = np.zeros((batch_size, state_size))
    return_actions = np.zeros((batch_size, action_size))
    return_rewards = np.zeros((batch_size, 1))
    return_next_states = np.zeros((batch_size, state_size))
    return_next_actions = np.zeros((batch_size, action_size))
    
#     print("replay_buffer[0].states.shape: {}".format(replay_buffer[0].states.shape))
#     print("replay_buffer[0].rewards[0]: {}".format(replay_buffer[0].rewards[0]))
#     print("replay_buffer[0].actions.shape: {}".format(replay_buffer[0].actions.shape))
#     print("replay_buffer[0].next_states.shape: {}".format(replay_buffer[0].next_states.shape))

    for i in range(batch_size):
        rand_frame_index = random.randint(0,len(replay_buffer)-2)
        return_states[i] = replay_buffer[rand_frame_index].states[0]
        return_actions[i] = replay_buffer[rand_frame_index].actions[0]
        return_rewards[i] = replay_buffer[rand_frame_index].rewards[0]
        return_next_states[i] = replay_buffer[rand_frame_index].next_states[0]
        return_next_actions[i] = replay_buffer[rand_frame_index+1].actions[0]
        #### TODO - make sure "next" actions don't roll over onto the next  playthrough.
        
    return return_states, return_actions, return_rewards, return_next_states, return_next_actions

# replay = []
# total_rewards = play_game(env, brain_name, replay, actor_model_local, add_noise=False)
# print("total rewards: {}".format(total_rewards))

# states, actions, rewards, next_states = getBatch(replay, 32)
# print("states: {}".format(states))
# print("actions: {}".format(actions))
# print("rewards: {}".format(rewards))
# print("next_states: {}".format(next_states))

In [ ]:
# train the model

epochs = 25 # 5
steps_per_epoch = 2000 # 1000 # 10
learning_iterations_per_step = 1 # 100
gamma = 0.99
tau = 0.001
batch_size = 128
epsilon_decay_batches = 50
epsilon_max = 0.05
epsilon_min = 0.0
current_state = reset_game(env, brain_name)

for epoch in range(epochs):
    total_actor_loss = 0.0
    total_critic_loss = 0.0    
    total_rewards = 0
    epsilon = max(epsilon_min, epsilon_max-epoch/epsilon_decay_batches)
    
    for game_step in range(steps_per_epoch):
        #play a game
        #total_rewards = play_game(env, brain_name, replay_buffer, actor_model_local)
        current_state = env_step(env, brain_name, current_state, replay_buffer, actor_model_local, epsilon, add_noise=True)
        total_rewards += np.sum(replay_buffer[-1].rewards)
        
        #if the game is done, reset and continue
        if np.any(replay_buffer[-1].dones):
            # new game
            current_state = reset_game(env, brain_name)
            current_state = env_step(env, brain_name, current_state, replay_buffer, actor_model_local, epsilon, add_noise=True)
            total_rewards += np.sum(replay_buffer[-1].rewards)
        
        if len(replay_buffer) < 4000:
            continue  
    
        #do some learning
        for learning_iteration in range(learning_iterations_per_step):
            states, actions, rewards, next_states, next_actions = getBatch(replay_buffer, batch_size)
            
            rewards_tensor = torch.from_numpy(rewards).float().cuda()
            # print("rewards_tensor.shape: {}".format(rewards_tensor.shape))
            next_actions_tensor = torch.from_numpy(next_actions).float().cuda()
            # print("next_actions_tensor.shape: {}".format(next_actions_tensor.shape))
            next_states_tensor = torch.from_numpy(next_states).float().cuda()
            # print("next_states_tensor.shape: {}".format(next_states_tensor.shape))
            states_tensor = torch.from_numpy(states).float().cuda()
            # print("states_tensor.shape: {}".format(states_tensor.shape))
            actions_tensor = torch.from_numpy(actions).float().cuda()
            # print("actions_tensor.shape: {}".format(actions_tensor.shape))
            
            # ---------------------------- update critic ---------------------------- #
            # Get predicted next-state actions and Q values from target models
            
            # Compute Q targets for current states (y_i)
            #rewards_tensor = rewards.unsqueeze(dim=1)
            Q_targets_next = critic_model_target(next_states_tensor, next_actions_tensor)
            # print("Q_targets_next: {}".format(Q_targets_next))            
            # print("Q_targets_next.shape: {}".format(Q_targets_next.shape))    
            Q_targets = rewards_tensor + (gamma * Q_targets_next)
            # print("Q_targets: {}".format(Q_targets))
            # print("Q_targets.shape: {}".format(Q_targets.shape))    
                
            # Compute critic loss
            Q_expected = critic_model_local(states_tensor, actions_tensor)
            # print("Q_expected: {}".format(Q_expected))
            # print("Q_expected.shape: {}".format(Q_expected.shape))   
            critic_loss = F.mse_loss(Q_expected, Q_targets)
            
            # Minimize the critic loss
            critic_optimizer.zero_grad()
            critic_loss.backward()
            torch.nn.utils.clip_grad_norm_(critic_model_local.parameters(), 1)
            critic_optimizer.step()
            total_critic_loss += critic_loss.item()

            # ---------------------------- update actor ---------------------------- #
            # Compute actor loss
            actions_pred = actor_model_local(states_tensor)
            actor_loss = -critic_model_local(states_tensor, actions_pred).mean()
            # print("states_tensor.shape: {}".format(states_tensor.shape))   
            # print("actions_pred.shape: {}".format(actions_pred.shape))   

            # Minimize the loss
            actor_optimizer.zero_grad()
            actor_loss.backward()
            actor_optimizer.step()
            total_actor_loss += actor_loss.item()
            #print("total_actor_loss: {}".format(total_actor_loss))

            # ----------------------- update target networks ----------------------- #
            #use very small Tau and update with every step
            soft_update_target(critic_model_local, critic_model_target, tau)
            soft_update_target(actor_model_local, actor_model_target, tau)
            
    print("epoch: {} - epsilon: {:.3f}, total_rewards: {:.3f}, critic_loss: {:.8f}, actor_loss: {:.6f}".format(epoch, epsilon, total_rewards, total_critic_loss/learning_iterations_per_step/steps_per_epoch, total_actor_loss/learning_iterations_per_step/steps_per_epoch))
    
torch.save(actor_model_local.state_dict(),  "actor_model_local.pt")
torch.save(actor_model_target.state_dict(), "actor_model_target.pt")
torch.save(critic_model_local.state_dict(), "critic_model_local.pt")
torch.save(critic_model_target.state_dict(),"critic_model_target.pt")


epoch: 0 - epsilon: 0.050, total_rewards: 50.070, critic_loss: 0.00000000, actor_loss: 0.000000
epoch: 1 - epsilon: 0.030, total_rewards: 61.240, critic_loss: 0.00001685, actor_loss: -0.002419
epoch: 2 - epsilon: 0.010, total_rewards: 68.560, critic_loss: 0.00196865, actor_loss: -1.256744
epoch: 3 - epsilon: 0.000, total_rewards: 51.980, critic_loss: 0.00110789, actor_loss: -1.305130
epoch: 4 - epsilon: 0.000, total_rewards: 50.770, critic_loss: 0.00102099, actor_loss: -1.323118
epoch: 5 - epsilon: 0.000, total_rewards: 62.250, critic_loss: 0.00106707, actor_loss: -1.387067
epoch: 6 - epsilon: 0.000, total_rewards: 53.380, critic_loss: 0.00115574, actor_loss: -1.455566
epoch: 7 - epsilon: 0.000, total_rewards: 53.050, critic_loss: 0.00138919, actor_loss: -1.492215


In [66]:
#play a game without noise
replay = []
total_rewards = 0.0

current_state = reset_game(env, brain_name)
epsilon = 0.0

for i in range(2000):
    current_state = env_step(env, brain_name, current_state, replay, actor_model_local, epsilon, add_noise=True)
    total_rewards += np.sum(replay[-1].rewards)
    #print("replay[-1].states: {}".format(replay[-1].states))
    if np.any(replay[-1].dones):
        print("new game!!! - {}".format(replay[-1].step_number))
        
        
print("total_rewards: {}".format(total_rewards))
            

done!!!
new game!!! - 1000
total_rewards: 0.4899999890476465


In [27]:

torch.save(actor_model_local.state_dict(),  "actor_model_local.pt")
torch.save(actor_model_target.state_dict(), "actor_model_target.pt")
torch.save(critic_model_local.state_dict(), "critic_model_local.pt")
torch.save(critic_model_target.state_dict(),"critic_model_target.pt")


In [55]:
for rb in replay_buffer:
    print(rb.step_number)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [62]:
for rep in replay_buffer:
    if np.any(rep.dones):
        print("new game!!!")